In [1]:

import numpy as np
import matplotlib.pyplot as plt
from pyuvdata import UVData
from pyuvdata import uvutils
from hera_qm import ant_metrics
import hera_cal
from copy import deepcopy
import uvtools

from matplotlib.colors import SymLogNorm
import sys
import os
from hera_qm import xrfi as xrfi
import numpy.ma as ma
import glob
import math


In [2]:


data_file_list = ['/data6/HERA/data/2458042/zen.2458042.12552.xx.HH.uv','/data6/HERA/data/2458042/zen.2458042.12552.yy.HH.uv','/data6/HERA/data/2458042/zen.2458042.12552.yx.HH.uv','/data6/HERA/data/2458042/zen.2458042.12552.xy.HH.uv']

uv = UVData()

uv.read_miriad(data_file_list)



Combined polarizations are not evenly spaced. This will make it impossible to write this data out to some file types.


In [3]:

aa = hera_cal.utils.get_aa_from_uv(uv)

info = hera_cal.omni.aa_to_info(aa)

reds = info.get_reds()


In [4]:

am = ant_metrics.Antenna_Metrics(data_file_list, reds, fileformat = 'miriad')


In [5]:

v = deepcopy(uv)


In [6]:

bases = []

lst = uv.get_antpairpols()

for i in range(uv.Nbls):
    
    pair = lst[4*i]
    
    bases.append((pair[0],pair[1]))


In [7]:

nonreds = []

for i in bases:
    
    flag = 0
    
    for j in reds:
        
        for k in j:
            
            if i == k:
                
                flag = 1
                
    if flag == 0:
        
        nonreds.append(i)


In [8]:

print uv.antenna_positions

print uv.antenna_numbers


[[  28.54149635  -86.35600842   -8.04679428]
 [   0.81464083  -50.05629453  -29.82931721]
 [ -14.37481942  -63.70823565  -62.64103936]
 [  11.36289375  -77.30163178  -29.87911091]
 [  12.16759751 -131.75416811  -62.87519242]
 [  15.43729238 -122.67655585  -51.89927641]
 [  20.73930714 -136.28828108  -51.94197718]
 [  46.63678713 -149.81897474  -19.28204167]
 [  17.95328508  -59.12635392   -7.97149171]
 [  12.64313462  -45.51782375   -7.92359747]
 [  29.35423193 -140.80539348  -41.04799869]
 [  32.63513192 -131.72334248  -30.09088496]
 [  33.85156678  -99.9645706    -8.09463731]
 [  21.20681292  -50.05508841    3.01474104]
 [  24.03622569 -127.19994404  -40.99508914]
 [  27.30110754 -118.12417964  -30.02774965]
 [  23.24740407  -72.74117587   -8.00915153]
 [  18.71019041 -113.59764582  -40.9370538 ]
 [  13.3761261   -99.99849888  -40.87389266]
 [   8.08204708  -86.38366017  -40.83625807]
 [   2.79594373  -72.76569153  -40.80371512]
 [  -2.42616744  -59.12260896  -40.81202489]
 [  16.672

In [9]:

a = uv.antenna_positions[24]
b = uv.antenna_positions[23]

dist = np.linalg.norm(a-b)

print dist


14.607952464


In [10]:

c = float(3e8)

B = float(uv.freq_array[0][-1]-uv.freq_array[0][0])

del_tau = 1./ B

tau = dist / c

print tau/del_tau


4.86456229515


In [11]:
T = uv.Ntimes*10
    
del_nu = 1. / T
    
earth_rot = 2*np.pi / 86164.09053
    
freq_naught = np.median(uv.freq_array[0])
    
nu = earth_rot * dist * freq_naught / c

print nu/del_nu

0.319464619081


In [12]:

def signal(amp,width,baseline):
    
    ant_1 = baseline[0]
    
    ant_2 = baseline[1]
    
    ant_1_ind = np.where(uv.antenna_numbers==ant_1)
    
    ant_2_ind = np.where(uv.antenna_numbers==ant_2)
    
    dist = np.linalg.norm(uv.antenna_positions[ant_1_ind]-uv.antenna_positions[ant_2_ind])
    
    c = float(3e8)
    
    B = float(uv.freq_array[0][-1]-uv.freq_array[0][1])
    
    del_tau = 2.*np.pi / B
    
    tau = dist / c
    
    temp = np.zeros((60,1024,4))
    
    T = uv.Ntimes*10
    
    del_nu = 2.*np.pi / T
    
    earth_rot = 2*np.pi / 86164.09053
    
    freq_naught = np.median(uv.freq_array[0])
    
    nu = earth_rot * dist * freq_naught / c
    
    for i in range(uv.Ntimes/2):
        
        for j in range(int(math.ceil(float(tau/del_tau)))):
            
            temp[i][j][0]= np.random.normal(amp,width) * np.exp(-i*(del_nu/nu))
            
            if (i != 0):
            
                temp[-i][j][0] = np.random.normal(amp,width) * np.exp(-i*(del_nu/nu))
            
    result = np.fft.fftn(temp)
    
    return result
